In [1]:
#stl
import os
import warnings

#data handling
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

#stats
import scipy
import sklearn

#network
import networkx as nx

#vis
import matplotlib.pyplot as plt
import seaborn as sns

#torch
import h5py
import anndata
import scanpy as sc

sns.set(font_scale = 1.5)
sns.set_theme()

# Data Preprocessing

**Output: multiple values of shared genes between all patients**

0. Find downstream coding protein genes
1. Find gene intersection list between all patients
2. Find cell type intersection list between all patients
3. Find gene intersection list in cell type intersections between all patients

## Data Filtering Assay

https://medium.com/@adam.nik/scrna-filtering-and-normalization-with-scanpy-adeae2577862

In [2]:
adata = anndata.read_h5ad("3716fb19-cedd-4fe5-abc4-5dbeb007fb65.h5ad")
adata

AnnData object with n_obs × n_vars = 881081 × 33145
    obs: 'Sample', 'donor_id', 'Region_x', 'Primary.Genetic.Diagnosis', 'n_genes', 'n_counts', 'percent_mito', 'percent_ribo', 'scrublet_score_z', 'scrublet_score_log', 'solo_score', 'cell_states', 'Assigned', 'self_reported_ethnicity_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'sex_ontology_term_id', 'assay_ontology_term_id', 'organism_ontology_term_id', 'is_primary_data', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'suspension_type', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length'
    uns: 'cell_states_colors', 'cell_type_ontology_term_id_colors', 'citation', 'leiden', 'neighbors', 'pca', 'schema_reference', 'schema_version', 'title', 'umap'
    obsm: 'X_pca', 'X_umap'

In [ ]:
#convert to gpu
import rapids_singlecell as rsc
rsc.get.anndata_to_GPU(adata=adata) 
cudata

**Quality Control based on Ribosomal or Mitochondrial Cells**

In [3]:
# mitochondrial genes
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo"], inplace=True, percent_top=[20], log1p=True
)